In [ ]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import re
import pandas as pd

# Ejercicio 1: Scraping de noticias

Código Python para extraer:

  - Titulares
  - Enlaces
  - Noticia completa
  - Categoría de la noticia
  - Fecha de creación

De las noticias presentes en el home de https://www.infosalus.com/

In [ ]:
def scrape_news_data(url):
    # Realizar una solicitud GET a la página principal
    response = requests.get(url)

    # Verificar que la solicitud sea exitosa (código de estado 200)
    if response.status_code != 200:
        raise Exception(f"Error al acceder a la URL: {url}")

    # Crear una instancia de BeautifulSoup para analizar la página
    soup = BeautifulSoup(response.text, "html.parser")

    # Encontrar todos los elementos <article> con la clase especificada
    articles = soup.find_all('article', class_='home-articulo-portada')

    # Crear una lista para almacenar los datos de cada noticia
    news_data = []

    # Expresiones regulares para encontrar lo que está después de "dateCreated", "articleBody" y "articleSection"
    date_created_pattern = r'"dateCreated":\s*"(.*?)"'
    article_body_pattern = r'"articleBody":\s*"(.*?)"'
    article_section_pattern = r'"articleSection":\s*"(.*?)"'

    # Iterar a través de los elementos <article> para acceder a los enlaces y detalles de cada noticia
    for article in articles:
        h2_element = article.find('h2', class_='home-articulo-titulo')

        # Verificar si se encontró un título
        if h2_element:
            # Extraer el título y el enlace
            article_title = h2_element.text.strip()
            article_link = h2_element.find('a')['href']

            # Realizar una solicitud GET al enlace de la noticia
            article_response = requests.get(article_link)

            # Verificar que la solicitud sea exitosa (código de estado 200)
            if article_response.status_code != 200:
                continue  # Saltar esta noticia si no se puede acceder al enlace

            soup = BeautifulSoup(article_response.text, 'html.parser')

            # Encontrar el contenido JSON dentro del script
            script_content = soup.find('script', {'type': 'application/ld+json'}).text

            # Encuentra las coincidencias en el JSON (json.load() no funcionaba)
            date_created_match = re.search(date_created_pattern, script_content)
            article_body_match = re.search(article_body_pattern, script_content, re.DOTALL) # Tenemos varias líneas (re.DOTALL)
            article_section_match = re.search(article_section_pattern, script_content)

            # Extraer los datos de la noticia si se encontraron coincidencias
            if date_created_match and article_body_match and article_section_match:
                date_created = date_created_match.group(1)
                article_body = article_body_match.group(1)
                article_section = article_section_match.group(1)

                # Agregar los datos de la noticia a la lista
                news_data.append({
                    'Titular': article_title,
                    'Enlace': article_link,
                    'Texto': article_body,
                    'Sección de la noticia': article_section,
                    'Fecha': date_created
                })

    # Crear un DataFrame a partir de la lista de datos de noticias
    df = pd.DataFrame(news_data)
    return df

# Llamada a la función y almacenamiento del resultado en un DataFrame
url = "https://www.infosalus.com/"
news_df = scrape_news_data(url)

# Mostrar el DataFrame
news_df

,Titular,Enlace,Texto,Sección de la noticia,Fecha
0,Estas 'microacciones' que haces diariamente en...,https://www.infosalus.com/salud-investigacion/...,"MADRID, 7 Oct. (EUROPA PRESS) -\r\nPasar horas...",Investigación,2023-10-07T08:14:51 +02:00
1,¿Lees con poca luz? ¿Y en el móvil? ¿Ves la te...,https://www.infosalus.com/salud-investigacion/...,"MADRID, 7 Oct. (EDIZIONES) - \r\n&nbsp;&nbs...",Investigación,2023-10-07T07:59:56 +02:00
2,¿Se te ha agarrado el resfriado? Como con el C...,https://www.infosalus.com/salud-investigacion/...,"MADRID, 7 Oct. (EUROPA PRESS) - \r\n El f...",Investigación,2023-10-07T08:29:55 +02:00
3,Estos son los efectos secundarios asociados a ...,https://www.infosalus.com/farmacia/noticia-son...,"MADRID, 6 Oct. (EUROPA PRESS) - \r\n Los ...",Farmacia,2023-10-06T08:34:51 +02:00
4,Las sinfonías clásicas pueden provocar un efec...,https://www.infosalus.com/salud-investigacion/...,"MADRID, 6 Oct. (EUROPA PRESS) - \r\n El m...",Investigación,2023-10-06T08:19:49 +02:00
5,¿De qué depende que seas vulnerable a las dist...,https://www.infosalus.com/salud-investigacion/...,"MADRID, 6 Oct. (EUROPA PRESS) - \r\n La re...",Investigación,2023-10-06T08:04:50 +02:00
6,Uno de cada 10 bebés nace prematuramente en to...,https://www.infosalus.com/actualidad/noticia-c...,"MADRID, 6 Oct. (EUROPA PRESS) - \r\n Un i...",Actualidad,2023-10-06T11:35:52 +02:00
7,El 36% de las personas con intento de suicidio...,https://www.infosalus.com/salud-investigacion/...,"MADRID, 6 Oct. (EUROPA PRESS) - \r\n La a...",Investigación,2023-10-06T12:19:53 +02:00
8,Las plagas de chinches aumentaron en España un...,https://www.infosalus.com/salud-investigacion/...,"MADRID, 6 Oct. (EUROPA PRESS) - \r\n Las ...",Investigación,2023-10-06T17:03:36 +02:00
9,¿Qué pasaría si una de las 'Tres gracias' de R...,https://www.infosalus.com/asistencia/noticia-p...,"MADRID, 6 Oct. (EUROPA PRESS) - \r\n La F...",Asistencia,2023-10-06T14:10:12 +02:00


# Ejercicio 2: Extraer imágenes de Piterest

In [ ]:
def scrape_images(url, ruta):
    # Realizar una solicitud GET a la página principal
    response = requests.get(url)

    # Verificar que la solicitud sea exitosa (código de estado 200)
    if response.status_code != 200:
        raise Exception(f"Error al acceder a la URL: {url}")

    # Crear una instancia de BeautifulSoup para analizar la página
    soup = BeautifulSoup(response.text, "html.parser")

    # Encontrar todas las etiquetas 'img' y extraer los enlaces 'src'
    for i, img_tag in enumerate(soup.find_all('img')):
        src = img_tag.get('src')

        # Si se ha encontrado descargar y almacenar
        if src:
          urllib.request.urlretrieve(src, f"{ruta}/imagen{i}.jpg")

url = 'https://www.pinterest.es/maitebortrodenas/imagenes-de-valencia/'

scrape_images(url,'data')